# ToBERT

In [1]:
import joblib
import os
import optuna
import sys
import torch
import wandb

sys.path.insert(0, os.path.abspath('../'))

from rhetorical_roles_classification import (
    AbsoluteSinusoidalEmbedder,
    AutoTransformerOverBERTForTokenClassification,
    RelativeSinusoidalEmbedder,
    RhetoricalRolesDatasetForTransformerOverBERT
)
from rhetorical_roles_classification.train import train
from rhetorical_roles_classification.test import test_ToBERT

from config import (
    DATA_FOLDER,
    MODELS_FOLDER,
    HUGGING_FACE_BERT_MODEL,
    NUM_LABELS,
    WANDB_PROJECT,
    MAX_SEGMENT_LENGTH,
    label2rhetRole
)

In [2]:
MODEL_NAME = "ToBERT"

MAX_DOCUMENT_LENGTH = 248

DEVICE = "cuda:0"

wandb_login_success = wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gmarinowandb-8601. Use `wandb login --relogin` to force relogin


## Dataset Loading

In [3]:
train_dataset = RhetoricalRolesDatasetForTransformerOverBERT(
    data_filepath=os.path.join(DATA_FOLDER, "train.json"),
    max_document_length=MAX_DOCUMENT_LENGTH,
    max_segment_length=MAX_SEGMENT_LENGTH,
    tokenizer_model_name=HUGGING_FACE_BERT_MODEL
)
valid_dataset = RhetoricalRolesDatasetForTransformerOverBERT(
    data_filepath=os.path.join(DATA_FOLDER, "valid.json"),
    max_document_length=MAX_DOCUMENT_LENGTH,
    max_segment_length=MAX_SEGMENT_LENGTH,
    tokenizer_model_name=HUGGING_FACE_BERT_MODEL
)
test_dataset = RhetoricalRolesDatasetForTransformerOverBERT(
    data_filepath=os.path.join(DATA_FOLDER, "test.json"),
    max_document_length=MAX_DOCUMENT_LENGTH,
    max_segment_length=MAX_SEGMENT_LENGTH,
    tokenizer_model_name=HUGGING_FACE_BERT_MODEL
)

## Embedders

In [4]:
absolute_embedder = AbsoluteSinusoidalEmbedder(
    max_document_length=MAX_DOCUMENT_LENGTH,
    embedding_dimension=768
)
joblib.dump(absolute_embedder, os.path.join(MODELS_FOLDER, "absolute_embedder"))

relative_embedder = RelativeSinusoidalEmbedder(
    max_document_length=MAX_DOCUMENT_LENGTH,
    embedding_dimension=768
)
joblib.dump(relative_embedder, os.path.join(MODELS_FOLDER, "relative_embedder"))

100%|██████████| 248/248 [08:05<00:00,  1.96s/it]


['/nas/users/g.marino/GA/RhetoricalRolesClassification/ITA-RhetRoles/models/relative_embedder']

In [4]:
absolute_embedder = joblib.load(os.path.join(MODELS_FOLDER, "absolute_embedder"))
relative_embedder = joblib.load(os.path.join(MODELS_FOLDER, "relative_embedder"))

## Hyperparameter Tuning

In [6]:
hyperparameters = {
    "batch_size": 1,
    "epochs": 20,
    "num_warmup_steps": 0,
    "accum_iter": 3,
    "early_stopping": True,
    "early_stopping_patience": 2
}

def objective(trial):
    
    print("\n************************ Starting New Optuna Trial ************************\n")
    
    hyperparameters["lr"] = trial.suggest_float("lr", 5e-6, 5e-4)
    hyperparameters["weight_decay"] = trial.suggest_float("weight_decay", 1e-3, 1e-1)
    
    embedding_type = trial.suggest_categorical("embedding", ["abs", "rel"])
    embedder = absolute_embedder if embedding_type == "abs" else relative_embedder

    transformer_dropout = trial.suggest_float("transformer_dropout", 0.1, 0.7)
    transformer_dim_feedforward = trial.suggest_int("transformer_dim_feedforward", 50, 1000)
    transformer = torch.nn.Transformer(
        d_model=768,
        nhead=12,
        batch_first=True,
        dim_feedforward=transformer_dim_feedforward,
        activation="gelu",
        dropout=transformer_dropout,
        layer_norm_eps=1e-12,
        num_encoder_layers=2,
        num_decoder_layers=0
    ).encoder

    model = AutoTransformerOverBERTForTokenClassification(
        model_name=HUGGING_FACE_BERT_MODEL,
        embedder=embedder,
        transformer=transformer,
        num_labels=NUM_LABELS,
        max_document_length=MAX_DOCUMENT_LENGTH,
        max_segment_length=MAX_SEGMENT_LENGTH,
        device=DEVICE
    )
    model_id = f"{MODEL_NAME}-Trial{trial.number}"

    with wandb.init(project=WANDB_PROJECT, config=hyperparameters, name=model_id):

        wandb.config["embedding_type"] = embedding_type
        wandb.config["transformer_dropout"] = transformer_dropout
        wandb.config["transformer_dim_feedforward"] = transformer_dim_feedforward        

        valid_loss = train(
            model=model,
            destination_path=os.path.join(MODELS_FOLDER, model_id),
            train_dataset=train_dataset,
            valid_dataset=valid_dataset,
            device=DEVICE,
            optuna_trial=trial,
            use_wandb=True,
            **hyperparameters
        )
        
    return valid_loss

In [5]:
study_name = f"{MODEL_NAME}.optuna"
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="minimize")

[I 2023-04-29 12:28:20,736] Using an existing study with name 'ToBERT.optuna' instead of creating a new one.


In [8]:
study.optimize(objective, n_trials=32)


************************ Starting New Optuna Trial ************************



Some weights of the model checkpoint at dlicari/Italian-Legal-BERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dlicari/Italian-Legal-BERT and


************** Training Started **************

EPOCH N. 1


1045it [05:22,  3.24it/s]



TRAIN RESULTS
Loss: 0.3934962834752918
Accuracy: 0.8441745574310415
MCC: 0.7552118255324357
Macro F1: 0.8423899720402994
Macro Precision: 0.8514664534728306
Macro Recall: 0.834224906053006
Micro F1: 0.8441745574310415
Micro Precision: 0.8441745574310415
Micro Recall: 0.8441745574310415

VALIDATION RESULTS
Loss: 0.2706819474697113
Accuracy: 0.8953222453222454
MCC: 0.837457054748137
Macro F1: 0.8952450228599662
Macro Precision: 0.9198564530995939
Macro Recall: 0.8748561350057319
Micro F1: 0.8953222453222454
Micro Precision: 0.8953222453222454
Micro Recall: 0.8953222453222454
--------------------------------------------------
EPOCH N. 2


...


TRAIN RESULTS
Loss: 0.21295778330038947
Accuracy: 0.9227195200846909
MCC: 0.8793525415752839
Macro F1: 0.9199651085823849
Macro Precision: 0.924365998937245
Macro Recall: 0.9157467479689163
Micro F1: 0.9227195200846909
Micro Precision: 0.9227195200846909
Micro Recall: 0.9227195200846909

VALIDATION RESULTS
Loss: 0.1408262997865677
Accuracy: 0.9418918918918919
MCC: 0.9110942758556412
Macro F1: 0.9326668484830986
Macro Precision: 0.9504680184721668
Macro Recall: 0.9191863637603748
Micro F1: 0.9418918918918919
Micro Precision: 0.9418918918918919
Micro Recall: 0.9418918918918919
--------------------------------------------------
EPOCH N. 3


1045it [04:59,  3.49it/s]



TRAIN RESULTS
Loss: 0.14513787230394298
Accuracy: 0.9515379638887255
MCC: 0.9244489863627919
Macro F1: 0.9517190507034672
Macro Precision: 0.955681364483007
Macro Recall: 0.9478620107124429
Micro F1: 0.9515379638887255
Micro Precision: 0.9515379638887255
Micro Recall: 0.9515379638887255

VALIDATION RESULTS
Loss: 0.12703166902065277
Accuracy: 0.9509355509355509
MCC: 0.9248124210320977
Macro F1: 0.9506521745423917
Macro Precision: 0.9628093371628863
Macro Recall: 0.9401907477456796
Micro F1: 0.9509355509355509
Micro Precision: 0.9509355509355509
Micro Recall: 0.9509355509355509
--------------------------------------------------


Epoch,▁▅█
Train Loss,█▃▁
Validation Loss,█▂▁
Epoch,3
Train Loss,0.14514
Validation Loss,0.12703


[I 2023-04-26 05:42:24,924] Trial 31 pruned. 


In [6]:
print("\n************************ Optuna Study Completed ************************\n")
best_trial = study.best_trial
print("Best trial:", best_trial.number)
print("Value:", best_trial.value)
print("Params:")
for key, value in best_trial.params.items():
    print(f"\t{key}: {value}")


************************ Optuna Study Completed ************************

Best trial: 23
Value: 0.06465253233909607
Params:
	embedding: rel
	lr: 8.32282596973597e-05
	transformer_dim_feedforward: 167
	transformer_dropout: 0.261449972781848
	weight_decay: 0.06927161013879751


## Test

In [7]:
best_trial = study.best_trial
best_params = best_trial.params

embedder = absolute_embedder if best_params["embedding"] == "abs" else relative_embedder

transformer = torch.nn.Transformer(
    d_model=768,
    nhead=12,
    batch_first=True,
    dim_feedforward=best_params["transformer_dim_feedforward"],
    activation="gelu",
    dropout=best_params["transformer_dropout"],
    layer_norm_eps=1e-12,
    num_encoder_layers=2,
    num_decoder_layers=0
).encoder

model = AutoTransformerOverBERTForTokenClassification(
    model_name=HUGGING_FACE_BERT_MODEL,
    embedder=embedder,
    transformer=transformer,
    num_labels=NUM_LABELS,
    max_document_length=MAX_DOCUMENT_LENGTH,
    max_segment_length=MAX_SEGMENT_LENGTH,
    device=DEVICE
)
model_id = f"{MODEL_NAME}-Trial{best_trial.number}"
model.load_state_dict(torch.load(os.path.join(MODELS_FOLDER, model_id)))

Some weights of the model checkpoint at dlicari/Italian-Legal-BERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dlicari/Italian-Legal-BERT and

<All keys matched successfully>

In [9]:
test_ToBERT(
    model=model,
    test_dataset=test_dataset,
    label2rhetRole=label2rhetRole,
    device=DEVICE,
)

100%|██████████| 294/294 [01:07<00:00,  4.36it/s]

Overall
	Accuracy: 0.9817366579177603
	MCC: 0.9720783528395587
	Macro F1: 0.9795379129594168
	Macro Precision: 0.9788718561392054
	Macro Recall: 0.980232328631341
	Micro F1: 0.9817366579177603
	Micro Precision: 0.9817366579177603
	Micro Recall: 0.9817366579177603
Rhetorical role: UNK
	Macro F1: 0.9944166802993666
	Macro Precision: 0.9941546830468946
	Macro Recall: 0.9946789875086139
Rhetorical role: CONCLUSIONI DELLE PARTI
	Macro F1: 0.9895915345087756
	Macro Precision: 0.9864074958850024
	Macro Recall: 0.9928325173843624
Rhetorical role: SVOLGIMENTO DEL PROCESSO
	Macro F1: 0.9836025201142898
	Macro Precision: 0.9835248845153817
	Macro Recall: 0.9836802133221133
Rhetorical role: MOTIVI DELLA DECISIONE
	Macro F1: 0.9863127553133663
	Macro Precision: 0.9862693161305649
	Macro Recall: 0.9863596044340375
Rhetorical role: P.Q.M.
	Macro F1: 0.9809305944204749
	Macro Precision: 0.9825436078002614
	Macro Recall: 0.9793306705903755
